## few aggregations has been done according to requirements 

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
race_results_df = spark.read.format('parquet').load(f"{gold_folder_path}/race_results")

In [0]:
race_results_df.display()

In [0]:
demo_df.select(countDistinct(col("race_name"))).display()

In [0]:
demo_df.filter(col("drivers_name") == 'Lewis Hamilton').select(sum('points')).display()

In [0]:
demo_df.groupBy("drivers_name").agg(sum("points")\
    ,avg("points")).display()

In [0]:
demo_df = race_results_df.filter(col("race_year").isin(2020, 2019))

In [0]:
from pyspark.sql.window import *

In [0]:
demo_df.withColumn('new', rank().over(Window.partitionBy('race_year').orderBy('points'))).display()

In [0]:
driver_standing_df = race_results_df.groupBy("race_year", "drivers_name", "nationality", "team_name")\
    .agg(sum("points").alias('total_points')\
        ,count(when(col("position") == 1, True)).alias("wins"))\
            .withColumn('rank', rank().over(Window.partitionBy(col('race_year')).orderBy(col('total_points').desc(),col('wins').desc())))

In [0]:
driver_standing_df.filter(col("race_year") == 2020).display()

In [0]:
driver_standing_df.write.format('parquet').mode('overwrite').save(f"{gold_folder_path}/driver_standings")

In [0]:
### constructors f1standings

In [0]:
race_results_df.display()

In [0]:
constructor_standings_df = race_results_df.groupBy("team_name", "race_year").agg(sum("points").alias("total_points"),count(when(col("position") == 1, True)).alias('wins'))\
    .withColumn('rank', rank().over(Window.partitionBy('race_year').orderBy(col("total_points").desc(),col("wins").desc())))

In [0]:
constructor_standings_df.filter(col("race_year") == 2020).display()

In [0]:
constructor_standings_df.write.mode('overwrite').format('parquet').save(f"{gold_folder_path}/constructor_standings")